In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1" # select which GPU(s) to use

In [2]:
import numpy as np
import math
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from sklearn.utils import class_weight

import itertools
print(tf.__version__)

2.4.0


In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    assert tf.config.experimental.get_memory_growth(physical_devices[0])
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

In [4]:
train_dir = './datasets/train/'
validate_dir = './datasets/validate/'
datasample_period = 60
prediction_period = 10
feature_columns = 40
band_size = 0.001
capacity_const = 6 * 40 * 60000 # calibration depends on single test run
n_train_files = len(os.listdir(train_dir))
n_validate_files = len(os.listdir(validate_dir))
steps_per_epoch = 0 # to be determined from generator

In [5]:
# Prepare a directory to store all the checkpoints.
checkpoint_dir = './models/checkpoint'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

In [6]:
def generate_data(directory, sample_size, prediction_period, feature_num, band_size):
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith((".npy")):
                # data = np.load(os.path.join(subdir, file))[:temp]
                data = np.load(os.path.join(subdir, file))
                # generate X, Y
                shape = data.shape
                X = np.zeros((shape[0]-sample_size, sample_size, feature_num), dtype=np.float16)
                Y = np.zeros(shape=(shape[0]-sample_size, 1), dtype=np.int)
                for i in range(shape[0]-sample_size):
                    # take the first feature_num columns as features
                    X[i] = data[i:i+sample_size, 1:feature_num+1]
                    delta_last = (data[i+sample_size-1, 0] - data[i, 0]) / data[i+sample_size-1, 0]
                    if delta_last < -band_size:
                        Y[i] = 0
                    elif delta_last > band_size:
                        Y[i] = 2
                    else:
                        Y[i] = 1
                # add the 4th dimension: 1 channel
                X = X.reshape(X.shape[0], sample_size, feature_num, 1)
                
                # calculate sample_weights for Y
                sample_weights_y = np.append(Y.flatten(), [0,1,2]) # to ensure exhaustive coverage
                sample_weights_categories = class_weight.compute_class_weight('balanced', classes = np.unique(sample_weights_y), y=sample_weights_y)
                idx = 0
                sample_weights = np.zeros(shape[0]-sample_size)
                for y in Y.flatten(): 
                    sample_weights[idx] = sample_weights_categories[y]
                    idx += 1

                # transform y to categorical arrays
                y_labels = to_categorical(sample_weights_y)[:-3]
                
                # stratify into batches
                total_size = np.prod(X.shape)
                batch_size = math.floor(total_size / capacity_const) + 1
                batches_x = np.array_split(X, batch_size)
                batches_y = np.array_split(y_labels, batch_size)
                batches_sample_weights = np.array_split(sample_weights, batch_size)
                for n in range(batch_size):
                    yield batches_x[n], batches_y[n], batches_sample_weights[n]

In [7]:
batch = generate_data(train_dir, datasample_period, prediction_period, feature_columns, band_size)

In [8]:
# calculate how many steps per epoch required
steps_per_epoch = sum(1 for dummy in batch)
print(steps_per_epoch, 'steps per epoch identified')

240 steps per epoch identified


In [9]:
# dataset = next(batch)
# dataset

In [10]:
# dataset[0].shape

In [11]:
# dataset[1].shape

In [12]:
# dataset[2].shape

In [13]:
def make_model(datasample_period, feature_columns):
    input_tensor = Input(shape=(datasample_period,feature_columns,1))

    # convolutional filter is (1,2) with stride of (1,2)
    layer_x = layers.Conv2D(16, (1,2), strides=(1,2))(input_tensor)
    layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
    layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
    layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
    layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
    layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)

    layer_x = layers.Conv2D(16, (1,2), strides=(1,2))(layer_x)
    layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
    layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
    layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
    layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
    layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)

    layer_x = layers.Conv2D(16, (1,10))(layer_x)
    layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
    layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
    layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)
    layer_x = layers.Conv2D(16, (4,1), padding='same')(layer_x)
    layer_x = layers.LeakyReLU(alpha=0.01)(layer_x)

    # Inception Module
    tower_1 = layers.Conv2D(32, (1,1), padding='same')(layer_x)
    tower_1 = layers.LeakyReLU(alpha=0.01)(tower_1)
    tower_1 = layers.Conv2D(32, (3,1), padding='same')(tower_1)
    tower_1 = layers.LeakyReLU(alpha=0.01)(tower_1)

    tower_2 = layers.Conv2D(32, (1,1), padding='same')(layer_x)
    tower_2 = layers.LeakyReLU(alpha=0.01)(tower_2)
    tower_2 = layers.Conv2D(32, (5,1), padding='same')(tower_2)
    tower_2 = layers.LeakyReLU(alpha=0.01)(tower_2)  

    tower_3 = layers.MaxPooling2D((3,1), padding='same', strides=(1,1))(layer_x)
    tower_3 = layers.Conv2D(32, (1,1), padding='same')(tower_3)
    tower_3 = layers.LeakyReLU(alpha=0.01)(tower_3)

    layer_x = layers.concatenate([tower_1, tower_2, tower_3], axis=-1)

    # concatenate features of tower_1, tower_2, tower_3
    layer_x = layers.Reshape((datasample_period,96))(layer_x)

    # 64 LSTM units
    layer_x = LSTM(64)(layer_x)
    # The last output layer uses a softmax activation function
    output = layers.Dense(3, activation='softmax')(layer_x)
    model = Model(input_tensor, output)

    model.summary()
    model.initial_epoch = 0
    opt = tf.keras.optimizers.Adam(lr=0.01, epsilon=1) # learning rate and epsilon are the same as paper DeepLOB
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model

In [14]:
def make_or_restore_model(datasample_period,feature_columns):
    # Either restore the latest model, or create a fresh one
    # if there is no checkpoint available.
    checkpoints = [checkpoint_dir + '/' + name
                   for name in os.listdir(checkpoint_dir)]
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=os.path.getctime)
        print('Restoring from', latest_checkpoint)
        model = load_model(latest_checkpoint)
        print(latest_checkpoint)
        model.initial_epoch = int(latest_checkpoint[latest_checkpoint.index("epoch=")+6:])
        return model
    print('Creating a new model')
    return make_model(datasample_period,feature_columns)

In [15]:
model = make_or_restore_model(datasample_period,feature_columns)

Creating a new model
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 60, 40, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 60, 20, 16)   48          input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 60, 20, 16)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 60, 20, 16)   1040        leaky_re_lu[0][0]                
_________________________________________________________________________

In [16]:
callbacks = [
    # This callback saves a SavedModel every 100 batches.
    # We include the training loss in the folder name.
    keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_dir + '/ckpt-loss={loss:.2f}-epoch={epoch:04d}/',
        save_freq='epoch', save_weights_only=False)
]

In [17]:
train_generator = generate_data(train_dir, datasample_period, prediction_period, feature_columns, band_size)
t1 = itertools.cycle(train_generator)
validate_generator = generate_data(validate_dir, datasample_period, prediction_period, feature_columns, band_size)
v1 = itertools.cycle(validate_generator)

In [18]:
initial_epoch = model.initial_epoch
# model.fit(train_generator, epochs=10, steps_per_epoch=n_train_files, validation_data=validate_generator, validation_steps=n_validate_files, callbacks=callbacks, initial_epoch=initial_epoch)
model.fit(t1, epochs=100, steps_per_epoch=steps_per_epoch, validation_data=v1, validation_steps=n_validate_files, callbacks=callbacks, initial_epoch=initial_epoch)

Epoch 1/10
240/240 [==============================] - 150s 595ms/step - loss: 1.1026 - accuracy: 0.4606 - val_loss: 1.0689 - val_accuracy: 0.3859
INFO:tensorflow:Assets written to: ./models/checkpoint/ckpt-loss=1.09-epoch=0001\assets
INFO:tensorflow:Assets written to: ./models/checkpoint/ckpt-loss=1.09-epoch=0001\assets
Epoch 2/10
240/240 [==============================] - 80s 333ms/step - loss: 1.0567 - accuracy: 0.4160 - val_loss: 0.9970 - val_accuracy: 0.3291
INFO:tensorflow:Assets written to: ./models/checkpoint/ckpt-loss=1.02-epoch=0002\assets
INFO:tensorflow:Assets written to: ./models/checkpoint/ckpt-loss=1.02-epoch=0002\assets
Epoch 3/10
240/240 [==============================] - 78s 326ms/step - loss: 0.9832 - accuracy: 0.3550 - val_loss: 0.9310 - val_accuracy: 0.3633
INFO:tensorflow:Assets written to: ./models/checkpoint/ckpt-loss=0.96-epoch=0003\assets
INFO:tensorflow:Assets written to: ./models/checkpoint/ckpt-loss=0.96-epoch=0003\assets
Epoch 4/10
240/240 [================

KeyboardInterrupt: 